In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

c:\Users\Usuario\Desktop\8-semestre\Tesis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Redes neuronales basdas en grafos

# Entities Exploration

## Concept

In [2]:
concept_df = pd.read_json("../data/MOOCCube/entities/concept.json",lines=True)

In [3]:
concept_df.shape

(114563, 4)

In [4]:
concept_df.sample(5)

id       name                            en  \
79592  K_瑞利-泰勒不稳定性_物理学  瑞利-泰勒不稳定性   Rayleigh-Taylor instability   
44614        K_紧开拓扑_数学       紧开拓扑          compactopen topology   
84628    K_高频点火装置_电气工程     高频点火装置       high frequency ignition   
55718      K_连续排污_机械工程       连续排污          continuous blow-down   
82711     K_半透明光阴极_电子学     半透明光阴极  semitransparent photocathode   

                                             explanation  
79592                       学科：物理学_等离子体物理学 见载：《物理学名词》第二版  
44614                 学科：数学_几何学 _拓扑学 _一般拓扑学 见载：《数学名词》第一版  
84628                      学科：电气工程_工业电热设备 见载：《电气工程名词》第一版  
55718  学科：机械工程_动力机械_锅炉_运行、维修和故障_运行 定义：锅筒锅炉运行中，为了保证锅水含...  
82711                         学科：电子学_真空电子学 见载：《电子学名词》第一版

## Course

In [5]:
course_df = pd.read_json("../data/MOOCCube/entities/course.json",lines=True)

In [6]:
course_df.shape

(706, 8)

In [7]:
course_df.sample(5)

id                 name  \
88        C_course-v1:TsinghuaX+30040323X+sp            水力学（自主模式）   
413      C_course-v1:UIR+2018040201X+2019_T1     大学生国家安全教育（2019春）   
296      C_course-v1:BNU+2017112001X+2019_T1  史学名家的治史历程与方法（2019春）   
525  C_course-v1:FUDANx+ECON13000701+2019_T1           经济学（2019春）   
523  C_course-v1:TsinghuaX+10430575X+2019_T1            力学（2019春）   

                                 prerequisites  \
88   选修过《微积分》或《高等数学》。如果对《理论力学》有所了解，会有助于本课程的学习。   
413                                          无   
296                                          无   
525                                    微积分基本知识   
523                       在中学物理和微积分基础上，即可学习本课程   

                                                 about  \
88   <p>《水力学》是水利、土木、环境、建筑和工程物理等相关专业的技术基础课，它以水为主要对象研...   
413  <p>\r\n\t《大学生国家安全教育》课程具有以下特色：一是课程内容丰富。课程以习近平总体...   
296  &nbsp; &nbsp; 《史学名家的治史历程与方法》课程由北京师范大学历史学院副院长姜海...   
525  经济学原理是经管类学生的一门基础核心课程，基本理论通常分为微观经济学与宏观经济学两大部分，其...   
523  <p>\n\t本课程是物理系基础物理的第一门课程。是本科生的核心基础课，主要面对需要坚实物理...   

                                     core_id  \
88        C_course-v1:TsinghuaX+30040323X+sp   
413      C_course-v1:UIR+2018040201X+2019_T1   
296      C_course-v1:BNU+2017112001X+2019_T1   
525  C_course-v1:FUDANx+ECON13000701+2019_T1   
523  C_course-v1:TsinghuaX+10430575X+2019_T1   

                                           video_order  \
88   [V_35b2877720324bc987a62b0c637e52c9, V_bdb9121...   
413  [V_cfc5a04fe5b348eaa11c90def13e40ca, V_93aaa75...   
296  [V_dd68104132cb472faf0082e269e5202c, V_9d3871c...   
525  [V_b425e1c02fba4852a0fd603331ab051f, V_838e935...   
523  [V_634d4b7f279b4c469be53c08221fb734, V_11fa34e...   

                                          display_name  \
88   [水力学课程介绍, 流动性和连续介质 前半段, 流动性和连续介质 后半段, 粘滞性, 压缩性...   
413  [Video, Video, Video, Video, Video, Video, Vid...   
296  [Video, Video, Video, Video, Video, Video, Vid...   
525  [Video, Video, Video, Video, Video, Video, Vid...   
523  [Video, Video, Video, Video, Video, Video, Vid...   

                                               chapter  
88   [01.01.01.01, 02.01.01.01, 02.01.02.01, 02.02....  
413  [01.01.01.01, 01.01.02.01, 01.01.03.01, 02.01....  
296  [01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04....  
525  [01.01.01.01, 01.01.02.01, 01.01.03.01, 01.01....  
523  [01.01.01.01, 01.01.02.01, 01.01.03.01, 01.01....

In [8]:
def remove_html_and_carriage_returns(text):
    soup = BeautifulSoup(text, "html.parser")
    text_without_html = soup.get_text().replace("\n","").replace("\r","").replace("\t","")
    
    return text_without_html

course_df["about"] = course_df["about"].apply(remove_html_and_carriage_returns)
course_df.sample(5)

id                       name  \
165        C_course-v1:TsinghuaX+THU201605X+sp  信息素养——学术研究的必修课（通识版）(自主模式)   
1    C_course-v1:TsinghuaX+THESIS2015X+2015_T1      2015年清华大学研究生学位论文答辩（二）   
622     C_course-v1:TsinghuaX+20190222+2019_T1                        病理学   
249             C_course-v1:CSU+2018020201X+sp                 主题英语（自主模式）   
80          C_course-v1:TsinghuaX+70150023X+sp                汽车理论 (自主模式)   

                    prerequisites  \
165  选修本课程者要求具备基本的计算机使用和网络资源访问能力。   
1                           无先修要求   
622           在学习本课程之前，需要先学习什么课程。   
249                             无   
80        微积分、理论力学、材料力学、汽车构造、振动理论   

                                                 about  \
165  “信息素养——学术研究的必修课（通识版）”是一门让学习者终身受益的关于方法与技能的课程，无论...   
1    学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平台，进...   
622  是主要从形态学角度探究疾病的医学基础学科。通过观察研究患病机体的形态变化，结合机能和代谢的改...   
249  本课程是大学生提升国际视野和跨文化交流能力的有益阶梯！本课程按主题分模块与你共同探讨英语学习...   
80   汽车理论课程是车辆工程和汽车工程专业同学的必修课，也可以为汽车行业人员深入了解汽车提供必备的...   

                                       core_id  \
165        C_course-v1:TsinghuaX+THU201605X+sp   
1    C_course-v1:TsinghuaX+THESIS2015X+2015_T1   
622     C_course-v1:TsinghuaX+20190222+2019_T1   
249             C_course-v1:CSU+2018020201X+sp   
80          C_course-v1:TsinghuaX+70150023X+sp   

                                           video_order  \
165  [V_8a41f50ab9a44a1faf31184edd88f6fc, V_49c73e7...   
1    [V_de0371575a9f4b5391c89ad16d68b5c2, V_d632034...   
622  [V_44ac4676992649338488d7bce8e00c66, V_a3f7ee1...   
249  [V_a1b0e1d042bf4244808e768c93112705, V_424bda4...   
80   [V_80cbacd44f0c48fb8c6420189265654a, V_a91ed56...   

                                          display_name  \
165  [认识信息素养, None, Video, Video, Video, Video, Vid...   
1    [答辩陈述, 答辩陈述, 问答及答辩结果, 答辩陈述, 问答及答辩结果, 答辩陈述, 问答及...   
622  [Video, Video, Video, Video, Video, Video, Vid...   
249  [Video, Video, Video, Video, Video, Video, Vid...   
80   [默认, 视频-1.1, 1.2-汽车的驱动力, 汽车的行驶阻力(1), 汽车的行驶阻力（2...   

                                               chapter  
165  [01.01.01.01, 01.01.01.02, 01.02.01.01, 01.03....  
1    [01.01.03.01, 01.02.03.01, 01.02.04.01, 01.03....  
622  [01.01.01.01, 02.01.01.01, 02.02.01.01, 02.03....  
249  [01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04....  
80   [01.01.01.01, 02.01.01.01, 02.02.01.01, 02.03....

In [68]:
# sentences = []
# course_ids = [] 
# for index, row in course_df.iterrows():
#     course_ids.append(row["id"])
#     sentences.append(row["about"])
# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
# embeddings = model.encode(sentences)


In [69]:
# course_df = course_df.explode('video_order').reset_index()
# course_df.head(5)

In [70]:
# course_df = course_df.drop('index', axis=1)
# course_df.head(5)

## Papers

In [71]:
paper_df = pd.read_json("../data/MOOCCube/entities/paper.json",lines=True)

In [72]:
paper_df.shape

(679790, 14)

In [73]:
paper_df.sample(5)

id name  \
51106   P_53e9a447b7602d9702d60c3c        
36551   P_53e9a7c8b7602d9703105210        
171067  P_55a46e2dc91b587b097b77cc        
523465  P_56d8514edabfae2eee050d8a        
529790  P_56d906b2dabfae2eeefaeae2        

                                                 abstract  \
51106   Dynamic response studies of a tension leg plat...   
36551   Web services are becoming more and more comple...   
171067                                                      
523465                                                      
529790                                                      

                                                  authors  \
51106   [{'id': '53f4558edabfaec22bb02c96', 'name': 'S...   
36551   [{'id': '53f46a2fdabfaeecd6a1bb8d', 'name': 'M...   
171067  [{'id': '562dfa2a45ce1e5967b995f7', 'name': 'C...   
523465  [{'name': 'monica ruscil'}, {'name': 'andrew e...   
529790                             [{'name': 'v szalay'}]   

                                  doi lang  num_citation  \
51106   10.1016/S0167-6105(97)00238-9   en            41   
36551             10.1007/11914853_25   en            48   
171067   10.1016/j.revmed.2008.03.371   en             0   
523465                                  en             5   
529790    10.1088/0305-4470/23/12/041   en             5   

                                                      pdf   sourcetype  \
51106                                                 NaN  publication   
36551   //static.aminer.org/pdf/PDF/000/524/425/web_se...  publication   
171067                                                     publication   
523465                                                NaN  publication   
529790                                                NaN  publication   

                                                    title  \
51106     Wind-induced response of a tension leg platform   
36551   Web service mining and verification of propert...   
171067                          A case of astasia abasia]   
523465     Animal feces collection and disposal apparatus   
529790  The overlap integral of associated Legendre fu...   

                                                    venue  year  \
51106   {'info': {'name': 'Journal of Wind Engineering...  1997   
36551   {'info': {'name': 'OTM Conferences (1)'}, 'iss...  2006   
171067  {'info': {'name': 'La Revue de médecine intern...  2009   
523465                        {'issue': '', 'volume': ''}  2010   
529790  {'info': {'name': 'Journal of Physics A'}, 'is...  1990   

                                 pages  \
51106   {'end': '240', 'start': '225'}   
36551   {'end': '425', 'start': '408'}   
171067  {'end': '175', 'start': '174'}   
523465        {'end': '', 'start': ''}   
529790        {'end': '', 'start': ''}   

                                                     urls  
51106   [http://dx.doi.org/10.1016/S0167-6105(97)00238-9]  
36551             [http://dx.doi.org/10.1007/11914853_25]  
171067  [http://www.ncbi.nlm.nih.gov/pubmed/18486281?r...  
523465                                                NaN  
529790    [http://dx.doi.org/10.1088/0305-4470/23/12/041]

## School

In [74]:
school_df = pd.read_json("../data/MOOCCube/entities/school.json",lines=True)

In [75]:
school_df.shape

(208, 3)

In [76]:
school_df.sample(5)

id        name  \
197     S_DelftX  荷兰代尔夫特理工大学   
166  S_TsinghuaX        清华大学   
146       S_SXPI  陕西工业职业技术学院   
171        S_SCU        苏州大学   
196     S_GYCTCM     贵州中医药大学   

                                                 about  
197  代尔夫特理工大学（Technische Universiteit Delft）位于荷兰代尔夫...  
166  清华大学（Tsinghua University），简称“清华”，由中华人民共和国教育部直属...  
146  陕西工业职业技术学院（Shaanxi Polytechnic Institute）地处历史文...  
171  苏州大学（Soochow University），简称“苏大”，坐落于历史文化名城苏州，国家...  
196  贵州中医药大学位于贵州省贵阳市，是贵州省人民政府举办的全日制普通高等学校，是国家中医药管理局...

## Teacher

In [77]:
teacher_df = pd.read_json("../data/MOOCCube/entities/teacher.json",lines=True)

In [78]:
teacher_df.shape

(1738, 3)

In [79]:
teacher_df.sample(5)

id name                                              about
1223  T_马洪兵  马洪兵  马洪兵，副教授。从事遥感图像处理与应用研究。专业核心课《数字逻辑与处理器基础》主讲教师之一，...
996   T_张小洁  张小洁  张小洁，女，副教授，硕士。陕西工业职业技术学院教师，2000年获得宝鸡文理学院电气工程及其自...
1612  T_雷丽萍  雷丽萍  雷丽萍，清华大学机械工程系，副研究员。主要从事材料加工中的数值模拟、塑性微成形、大型装备结构...
32    T_陈朝晖  陈朝晖  陈朝晖，博士，重庆大学土木工程学院教授，博士生导师，慕课《结构力学》主讲教师与负责人，重庆市...
172   T_李世新  李世新  李世新 ，北京理工大学人文与社会科学学院副教授，哲学博士。研究领域：工程伦理学、科学技术与社...

## User

In [80]:
user_df = pd.read_json("../data/MOOCCube/entities/user.json",lines=True)

In [81]:
user_df.shape

(199199, 4)

In [82]:
user_df.sample(5)

id name                                       course_order  \
50191   U_9618374  畅博达  [C_course-v1:TsinghuaX+AP000003X+2019_T1, C_co...   
33020   U_3386360  禹和韵  [C_course-v1:TsinghuaX+34100325_X+sp, C_course...   
129036  U_8687918  淦永寿  [C_course-v1:CSU+210201T10+sp, C_course-v1:Tsi...   
155498   U_362095  柳夜山  [C_course-v1:TsinghuaX+30240243X+sp, C_course-...   
99733   U_7921634  刘敏叡  [C_course-v1:TsinghuaX+10610224X+sp, C_course-...   

                                              enroll_time  
50191   [2019-03-03 09:38:42, 2018-11-19 09:37:58, 201...  
33020   [2016-10-22 10:02:39, 2016-07-05 20:35:11, 201...  
129036         [2019-02-21 12:45:09, 2018-03-14 20:23:05]  
155498  [2019-01-06 20:32:40, 2019-01-11 23:36:27, 201...  
99733   [2017-11-04 10:31:45, 2017-11-04 10:32:57, 201...

## Video

In [83]:
video_df = pd.read_json("../data/MOOCCube/entities/video.json",lines=True)

In [84]:
video_df.shape

(38181, 5)

In [85]:
video_df.sample(5)

id                  name  \
19505  V_db67e92a64c940a58cd349ff2657a645            汽车燃油经济性的评价   
27490  V_3c91b77074ae47158837cf8b9f326e07                 Video   
16789    V_L7V1_OpenGL_Shading_Motivation  L7V1: OpenGL 着色：学习动机   
28173  V_9831eee7251b4e5fa287d6c1c47a79fc     爱奇艺用户体验设计中心负责人-李威   
19478  V_6014324dd21b4cbb99cfd80139198f51                 Video   

                                                   start  \
19505  [14129, 15705, 17201, 19513, 21593, 24721, 270...   
27490  [22550, 23850, 25300, 26475, 28025, 30200, 322...   
16789  [650, 6000, 11629, 13740, 18990, 23849, 29230,...   
28173  [9490, 12669, 14660, 19407, 21769, 25003, 2669...   
19478  [28166, 39984, 43694, 51458, 62783, 69151, 824...   

                                                     end  \
19505  [15517, 17017, 19329, 21393, 24569, 26841, 288...   
27490  [23850, 25300, 26475, 28025, 29750, 32200, 343...   
16789  [6000, 11629, 13740, 18990, 23849, 29230, 3453...   
28173  [12163, 14331, 19072, 20557, 24641, 26494, 284...   
19478  [35095, 42700, 49209, 61169, 67876, 79653, 852...   

                                                    text  
19505  [同学们大家好, 从这一节课开始, 我们介绍汽车理论第二章, 汽车的燃油经济性, 下面呢介绍...  
27490  [这一节我们介绍, 变速器的换挡方式, 变速器我们知道, 都是用齿轮来获得, 所需要的传动比...  
16789  [这一讲，我们将学习OpenGL中的着色，它对完成作业2很关键，, 同时会引起你对任何对场景...  
28173  [我说一下我对目前企业的一些看法, 就是以我现在所在的爱奇艺为例, 现在企业越来越重视用户体...  
19478  [第14課、会話です。まず、聞いてください//第14课 会话部分 首先 请听一遍, ちょっと...

### Relacion Curso-Concepto

In [9]:
# course_concept_df = pd.read_json("../data/MOOCCube/relations/course-concept.json",lines=True)
with open("../data/MOOCCube/relations/course-concept.json", "r",encoding="utf-8") as file:
    courses_ids = []
    concept_ids = []
    for line in file:
        line = line.split("\t")
        courses_ids.append(line[0])
        concept_ids.append(line[1].replace("\n",""))
        

In [10]:
course_concept_relation_df = pd.DataFrame({"id_course":courses_ids,"id_concept":concept_ids})

In [11]:
course_concept_relation_df.sample(5)

id_course     id_concept
147576        C_course-v1:SichuanU+304231020+sp    K_业主_管理科学技术
109127      C_course-v1:ZZU+2018032501X+2019_T1    K_价值_管理科学技术
78326            C_course-v1:RiceX+AdvBIO_4x+sp   K_生命系统_自然辩证法
156564  C_course-v1:TsinghuaX+AP000005X+2019_T1        K_正切_数学
24452             C_course-v1:GZHU+000001004+sp  K_工作计划_管理科学技术

In [12]:
course_concept_df = course_df.merge(course_concept_relation_df, left_on="id", right_on="id_course")
course_concept_df = course_concept_df.merge(concept_df, left_on="id_concept", right_on="id")
course_concept_df.head(5)

id_x                    name_x  \
0  C_course-v1:McGillX+ATOC185x+2015_T1                自然灾害（自主模式）   
1    C_course-v1:WellesleyX+HIST229x+sp          亚历山大大帝伟大吗？（自主模式）   
2         C_course-v1:UQx+TROPIC101x+sp           热带海岸线生态系统(自主模式）   
3         C_course-v1:MIL+MIL2016001+sp   我国周边14个陆地邻国基本情况介绍(自主模式)   
4    C_course-v1:TsinghuaX+30040352X+sp  地下水文学（水文学原理及应用（2））（自主模式）   

                prerequisites  \
0                           无   
1                           无   
2                 建议掌握高中理科知识。   
3               本课程为基础课，无先修要求   
4  建议先修土力学、水力学、水文学原理与应用（1）等课程   

                                               about  \
0  地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...   
1  亚历山大大帝征服了古希腊人所知的大部分世界，将他的帝国中的东西地区，不同种族的人民都融为一体...   
2  你是否想要学习保护热带海岸线生态系统的相关技能和知识？这些栖息地为数以亿计的人们提供了产品和...   
3         在亚欧大陆东部，太平洋西岸，矗立着世界上陆地边境线最长，邻国众多的国家----...   
4  课程目标是以地下含水介质中的非饱和－饱和地下水为对象，运用水文循环和水量平衡原理，深入理解其...   

                                core_id  \
0  C_course-v1:McGillX+ATOC185x+2015_T1   
1    C_course-v1:WellesleyX+HIST229x+sp   
2         C_course-v1:UQx+TROPIC101x+sp   
3         C_course-v1:MIL+MIL2016001+sp   
4    C_course-v1:TsinghuaX+30040352X+sp   

                                         video_order  \
0  [V_f6f710068b994452885b90e11b6ee5c5, V_7339568...   
1  [V_2fe925ce684049f68ca765f028f79f0b, V_e55a44c...   
2  [V_6911fde00bb24ababc2418f582972407, V_3d6a0f7...   
3  [V_697ab72b4d7547dca9b250cbf60e8318, V_dfff65d...   
4  [V_6b4fcc4310af4b3da028c357cf7e7f8a, V_dadda25...   

                                        display_name  \
0  [Video: Overview 1, Video: Overview 2, Video: ...   
1  [Lecture Module 1: Introductions, Conversation...   
2  [Introduction Video, Finding your way around t...   
3  [课程介绍, 吉尔吉斯授课视频, 尼泊尔授课视频, 课程讲授, 授课视频, 授课视频, 授课...   
4  [地下水文学-概论-1.1, 地下水文学-概论-1.2, Video, Video, Vid...   

                                             chapter  \
0  [01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02....   
1  [01.01.02.01, 01.01.05.01, 01.01.07.01, 01.01....   
2  [01.01.01.02, 01.03.01.02, 02.01.02.02, 02.02....   
3  [01.01.01.01, 02.01.01.01, 03.01.01.01, 04.01....   
4  [01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04....   

                              id_course id_concept      id_y name_y    en  \
0  C_course-v1:McGillX+ATOC185x+2015_T1   K_地带_地理学  K_地带_地理学     地带  zone   
1    C_course-v1:WellesleyX+HIST229x+sp   K_地带_地理学  K_地带_地理学     地带  zone   
2         C_course-v1:UQx+TROPIC101x+sp   K_地带_地理学  K_地带_地理学     地带  zone   
3         C_course-v1:MIL+MIL2016001+sp   K_地带_地理学  K_地带_地理学     地带  zone   
4    C_course-v1:TsinghuaX+30040352X+sp   K_地带_地理学  K_地带_地理学     地带  zone   

                                         explanation  
0  学科：地理学_地理学总论 定义：地理环境各组成成分及其相互作用形成的地理景观。在地表呈带状分...  
1  学科：地理学_地理学总论 定义：地理环境各组成成分及其相互作用形成的地理景观。在地表呈带状分...  
2  学科：地理学_地理学总论 定义：地理环境各组成成分及其相互作用形成的地理景观。在地表呈带状分...  
3  学科：地理学_地理学总论 定义：地理环境各组成成分及其相互作用形成的地理景观。在地表呈带状分...  
4  学科：地理学_地理学总论 定义：地理环境各组成成分及其相互作用形成的地理景观。在地表呈带状分...

In [23]:
concept_per_course_df = course_concept_df.groupby('id_course')['id_concept'].count().reset_index()
concept_per_course_df.head(5)

id_course  id_concept
0     C_course-v1:ACCA+FA1_X+2019_T1         117
1          C_course-v1:ACCA+FA1_X+sp         117
2  C_course-v1:ACCA+FA1_X_en+2019_T1          93
3     C_course-v1:ACCA+FA2_X+2019_T1         115
4          C_course-v1:ACCA+FA2_X+sp         115

In [26]:
course_per_concept_df = course_concept_df.groupby('id_concept')['id_course'].count().reset_index()
course_per_concept_df.sample(20)

id_concept  id_course
3243      K_凝固_材料科学技术          1
7520      K_套利_管理科学技术          7
24059      K_隔声罩_机械工程          1
19902         K_脐炎_医学          1
9764       K_微观粒子_物理学         21
7077         K_外斜视_医学          2
18913      K_终参数_机械工程          1
4983        K_反渗透膜_化学          1
6427      K_地下水资源_地质学          1
15997    K_生态价值_自然辩证法          9
22724         K_近场_化学          1
5760         K_吸附量_化学          5
19938       K_脱模_机械工程          3
19779       K_胎面_机械工程          1
16284     K_电子束蒸发_电子学          1
9022        K_平焊_机械工程          1
5153       K_变形力_机械工程          4
12585       K_本地话_语言学          1
12519  K_服务提供者_管理科学技术         19
4706     K_原始农业_自然辩证法          5

In [16]:
concept_per_course_mean = concept_per_course_df['id_concept'].mean()
course_per_concept_mean =course_per_concept_df['id_course'].mean()
print(concept_per_course_mean)
print(course_per_concept_mean)

245.6090775988287
6.6565215666045


In [22]:
concept_per_course_std = concept_per_course_df['id_concept'].std()
course_per_concept_std =course_per_concept_df['id_course'].std()
print(concept_per_course_std)
print(course_per_concept_std)

121.38267825448973
15.32440326226328


In [30]:
datos_ordenados = course_per_concept_df.sort_values(by="id_course")
datos_ordenados.tail(5)
a =course_per_concept_df["id_course"].value_counts()

In [31]:
print(a)

id_course
1      8448
2      4743
3      2678
4      1828
5      1260
       ... 
145       1
155       1
181       1
177       1
189       1
Name: count, Length: 179, dtype: int64
